In [ ]:
from chemcompute import chemcompute

In [ ]:
job = chemcompute.download(1221668)

In [ ]:
print (job)

In [ ]:
from jupyter_jsmol import JsmolView
view1 = JsmolView.from_file(job, inline=False) 
view1

In [ ]:
data = chemcompute.parse(job)

In [ ]:
# how does energy change as geometry converges
data.scfenergies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(data.scfenergies)
plt.xlabel("Step")
plt.ylabel("Energy / h")
plt.title("Geometry Optimization")
plt.grid(True)
plt.show()


In [ ]:
# how do the atoms move as geometry converges
data.atomcoords

In [ ]:
# which atom is which
# result:  atom 0: oxygen    atom 1: hydrogen    atom 2: hydrogen
data.atomnos

In [ ]:
# write a function to find the distance between two atoms
def distance(atom1,atom2):
    deltax = atom1[0] - atom2[0]
    deltay = atom1[1] - atom2[1]
    deltaz = atom1[2] - atom2[2]
    d = (deltax**2 + deltay**2 + deltaz**2)**(0.5)
    return d
    

In [ ]:
# how many geometry optimzation frames are there?  (includes starting geometry #0)
len(data.atomcoords)

In [ ]:
# distance between O and H in starting geometry
frame = 0

print (distance(data.atomcoords[frame][0], data.atomcoords[frame][1]))

In [ ]:
# optimized O-H bond length
frame = len(data.atomcoords)-1

print (distance(data.atomcoords[frame][0], data.atomcoords[frame][1]))

In [ ]:
# distance between O and H as the geometry converges
OHdistances = []
for frame in range(0,len(data.atomcoords)):
    OHdistances.append(distance(data.atomcoords[frame][0], data.atomcoords[frame][1]))
plt.plot(OHdistances)
plt.xlabel("Step")
plt.ylabel("O-H distance / A")
plt.title("Geometry Optimization")
plt.grid(True)
plt.show()

In [ ]:
# compute angle between H-O-H using law of cosines
from math import acos
def angle(atom1,atom2,atom3):
    a = distance(atom1,atom2)
    b = distance(atom3,atom2)
    c = distance(atom1,atom3)
    
    cosTheta = (a**2 + b**2 - c**2) / (2*a*b)
    theta = acos(cosTheta)
    return theta * 360 / (2*3.14159)
    
    

In [ ]:
# calculate H-O-H bond bond angle (initial)
# remember:  atom 0 is O, atom 1 is H, and atom 2 is H
frame = 0
angle(data.atomcoords[frame][1], data.atomcoords[frame][0], data.atomcoords[frame][2])

In [ ]:
# HOH bond angle as the geometry converges
HOHangles = []
for frame in range(0,len(data.atomcoords)):
    HOHangles.append(angle(data.atomcoords[frame][1], data.atomcoords[frame][0], data.atomcoords[frame][2]))
plt.plot(HOHangles)
plt.xlabel("Step")
plt.ylabel("H-O-H bond angle / degrees")
plt.title("Geometry Optimization")
plt.grid(True)
plt.show()

In [ ]:
#enthalpy (hartree / particle)
#units are here:  https://cclib.github.io/data.html
print (data.enthalpy)


In [ ]:
# convert to kJ / mol
# 1 h = 2625.5 kJ/mol   https://cccbdb.nist.gov/hartreex.asp
# remember this is not deltaHf, it's H
print (f"H = {round(data.enthalpy * 2625.5, 2)} kJ/mol")

In [ ]:
# IR frequencies (cm^-1)
print (data.vibfreqs)
# IR intensities (km / mol)
print (data.vibirs)

In [ ]:
# plot IR spectra (no linewidth from GAMESS, just infinitely thin lines)
for i in range(len(data.vibfreqs)):
    plt.vlines(data.vibfreqs[i],0,data.vibirs[i])


In [ ]:
# animate vibrations
view1.script("frame last; vibration on;")
# scroll up to the JSmol window

In [ ]:
# molecular orbital energies
print (data.moenergies[0])

In [ ]:
for energy in data.moenergies[0]:
    plt.hlines(energy, 0, 1)
plt.title("MO Energy diagram")
plt.ylabel("MO Energy / h")
plt.xticks([])
plt.show()
    